*This notebook gets the UDM ID from `ra_one_off` table, fetches the measurement data from RIPE and inserts the entire dataset into `ra_one_off_parameters`.*
- - - 

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
import sqlite3
import pandas as pd
import requests
import ipaddress
import time

In [4]:
def get_json_resource_from_absolute_uri(url, query_params):
    try: res = requests.get(url, params = query_params)
    except Exception as e: print(e, file=sys.stderr)
    else:
        try: res_json = res.json()
        except Exception as e: print(e, file=sys.stderr)
        else: 
            return res_json

In [5]:
def get_udm_data(udm):
    if udm is None: return None
    base_uri = 'https://atlas.ripe.net'; 
    url = '%s/api/v2/measurements/%s/results'%(base_uri, str(udm))
    try: res_json = get_json_resource_from_absolute_uri(url, 'format=json')
    except Exception as e: print(e, file=sys.stderr)
    else: return res_json

In [6]:
DB_LOCATION = 'lastmile.db'
RA_ONE_OFF = 'ra_one_off'
RA_ONE_OFF_PARAMETERS = 'ra_one_off_parameters'

In [7]:
con = sqlite3.connect(DB_LOCATION)

In [8]:
query = '''SELECT    o.msm_id as 'udmid'
                   , o.asn_v4 as 'source_asn'
           FROM    %s as o
        '''%(RA_ONE_OFF)

df = pd.read_sql(query, con)

In [9]:
df.count()

udmid         2751
source_asn    2751
dtype: int64

In [10]:
df.head()

,udmid,source_asn
0,2863204,7922
1,2863205,6830
2,2863206,12322
3,2863207,3320
4,2863208,3215


In [11]:
def get_df_udm_json_data(udm_json_data):
    
    df = pd.DataFrame(udm_json_data)

    df.rename (  columns={  'from'        :    'probe_ip'
                          , 'prb_id'      :    'probeid'
                          , 'size'        :    'packet_size'                      
                          , 'timestamp'   :    'starttime'                                            
                          , 'type'        :    'msm_type'                                            
                         }
               , inplace=True
              )
    
    df_result = pd.DataFrame(df['result'][0])
    df_result['probeid'] = df['probeid'][0]

    for index, row in df.iterrows():
        if index == 0: continue
        df_temp = pd.DataFrame(df['result'][index])
        df_temp['probeid'] = df['probeid'][index]
        df_result = df_result.append(df_temp)    
        
    del df['result']        
    
    df_joined = df.merge(df_result, on='probeid')
    
    try: 
        df_joined.drop('error', axis=1, inplace=True)
        print('*', end='')
    except Exception as e: pass
    
    df_joined.rename (  columns={  'hop'         :    'result_hop_no'
                                 , 'result'      :    'result_hop_blob'
                                }
                      , inplace=True
                     )
    
    try: df_joined['result_hop_blob'] = df_joined['result_hop_blob'].apply(str)
    except KeyError as e: df_joined['result_hop_blob'] = ''
        
    df_joined.dropna(  how='any'
                     , subset=['msm_id', 'probeid', 'endtime', 'result_hop_no']
                     , inplace=True
                    )
        
    return df_joined

In [12]:
def write_df_to_table(con, df, tablename):
    cur = con.execute('pragma foreign_keys=ON')
    
    index_label = ['msm_id', 'probeid', 'endtime', 'result_hop_no']
    df = df.set_index(index_label)
    
    df.to_sql(  '%s'%tablename
              , con
              , flavor='sqlite'
              , if_exists = 'append'
              , index_label = index_label
             )

    con.commit()

In [13]:
for index, row in df.iterrows():
    udmid, source_asn = row
    try:
        udm_json_data = get_udm_data(udmid)
        if len(udm_json_data) == 0: 
            print('^', end=""); continue
        df = get_df_udm_json_data(udm_json_data)
        write_df_to_table(con, df, RA_ONE_OFF_PARAMETERS)
    except Exception as e: print(e, udmid)
    else: print('.', end="")

..............................................................................................................*......................................................................................................................................................................................................................................................................................................................................................................................................................................*.................................................^.......................................................................................^.........................................^............................................................................................................................^........................................................................^.....................^........................................^..........^^.............

In [14]:
con.close()